In [1]:
import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

In [34]:
def SMAPE(label,y_pred):
    y_pred = y_pred.get_label()
    a= np.abs(y_pred-label)/(y_pred+label)/2
    b = np.sum(a)
    c = b/len(label)
    d = 40 * (1 - c / 2)
    return 'SMAPE',d,True

## 1.hyperopt对lightgbm自动调参

In [2]:
from hyperopt import fmin, tpe, hp, partial

# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         "num_trees": hp.randint("num_trees", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "bagging_fraction": hp.randint("bagging_fraction", 5),
         "num_leaves": hp.randint("num_leaves", 6),
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['num_trees'] = argsDict['num_trees'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["bagging_fraction"] = argsDict["bagging_fraction"] * 0.1 + 0.5
    argsDict["num_leaves"] = argsDict["num_leaves"] * 3 + 10
    if isPrint:
        print(argsDict)
    else:
        pass
    return argsDict

In [3]:
def lightgbm_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)

    params = {'nthread': -1,  # 进程数
              'max_depth': argsDict['max_depth'],  # 最大深度
              'num_trees': argsDict['num_trees'],  # 树的数量
              'eta': argsDict['learning_rate'],  # 学习率
              'bagging_fraction': argsDict['bagging_fraction'],  # bagging采样数
              'num_leaves': argsDict['num_leaves'],  # 终点节点最小样本占比的和
              'objective': 'regression',
              'feature_fraction': 0.7,  # 样本列采样
              'lambda_l1': 0,  # L1 正则化
              'lambda_l2': 0,  # L2 正则化
              'bagging_seed': 100,  # 随机种子,light中默认为100
              'silent':1,
              }
    #rmse
    params['metric'] = ['rmse']

    model_lgb = lgb.train(params, train_data, num_boost_round=300, valid_sets=[test_data],early_stopping_rounds=100,feval=SMAPE)

    return get_tranformer_score(model_lgb)

def get_tranformer_score(tranformer):

    model = tranformer
    prediction = model.predict(X_test, num_iteration=model.best_iteration)

    return mean_squared_error(y_test, prediction)

## 2.加载数据，构造lightgbm数据集

In [44]:
path='../../data/AD/da/'
train=pd.read_csv(path+"train1_without0_delete.csv",index_col=0)
testB_df_path = '../../data/AD/da/Btest_sample_new.dat'
test_name=['id','ad_id','create_time','ad_size','ad_trade_id','good_class','good_id','ad_acc_id','deliver_time','people','ad_bid']
testB_df = pd.read_csv(testB_df_path,delimiter = '\t',parse_dates = ['create_time'],header=None,names = test_name)

In [45]:
df_train=train[train.create_update_time!=20190318]
df_test=train[train.create_update_time==20190318]

In [46]:
df_train.drop(['create_update_time','deliver_time','people'], axis=1, inplace=True)
df_train.drop_duplicates(keep='first', inplace=True)
df_test.drop(['create_update_time','deliver_time','people'], axis=1, inplace=True)
df_test.drop_duplicates(keep='first', inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [47]:
y_train = df_train.label.values
y_test = df_test.label.values
X_train = df_train.drop('label', axis=1).values
X_test = df_test.drop('label', axis=1).values
train_data = lgb.Dataset(data=X_train,label=y_train)
test_data = lgb.Dataset(data=X_test,label=y_test)

In [17]:
train.head()

,ad_id,create_update_time,ad_size,ad_trade_id,good_class,good_id,ad_acc_id,deliver_time,people,ad_bid,label
0,31,20190312,40,224,13,18683,12577.0,"281474976694272,281474976694272,28147497669427...",area:11442,100,14.0
10,32,20190218,40,136,13,32534,18752.0,"70368475742208,70368475742208,70368475742208,7...",area:7572,83,1.0
11,32,20190219,40,136,13,32534,18752.0,"70368475742208,70368475742208,70368475742208,7...",area:7572,83,11.0
33,32,20190313,40,136,13,32534,18752.0,"70368475742208,70368475742208,70368475742208,7...",area:7572,82,4.0
38,32,20190318,40,136,13,32534,18752.0,"70368475742208,70368475742208,70368475742208,7...",area:7572,80,4.0


In [18]:
testB_df.head()

,id,ad_id,create_time,ad_size,ad_trade_id,good_class,good_id,ad_acc_id,deliver_time,people,ad_bid
0,1,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81276 entries, 0 to 650448
Data columns (total 11 columns):
ad_id                 81276 non-null int64
create_update_time    81276 non-null int64
ad_size               81276 non-null int64
ad_trade_id           81276 non-null int64
good_class            81276 non-null int64
good_id               81276 non-null int64
ad_acc_id             81276 non-null float64
deliver_time          81276 non-null object
people                81276 non-null object
ad_bid                81276 non-null int64
label                 81276 non-null float64
dtypes: float64(2), int64(7), object(2)
memory usage: 7.4+ MB


## 3.开始调参（可以不运行）

In [48]:
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(lightgbm_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

[1]	valid_0's rmse: 194.81	valid_0's SMAPE: 32.9992
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 192.601	valid_0's SMAPE: 33.0532
[3]	valid_0's rmse: 190.998	valid_0's SMAPE: 33.0984
[4]	valid_0's rmse: 189.771	valid_0's SMAPE: 33.1457
[5]	valid_0's rmse: 189.156	valid_0's SMAPE: 33.1911
[6]	valid_0's rmse: 188.45	valid_0's SMAPE: 33.2356
[7]	valid_0's rmse: 188.371	valid_0's SMAPE: 33.2706
[8]	valid_0's rmse: 187.428	valid_0's SMAPE: 33.3139
[9]	valid_0's rmse: 186.906	valid_0's SMAPE: 33.3529
[10]	valid_0's rmse: 187.113	valid_0's SMAPE: 33.3837
[11]	valid_0's rmse: 187.23	valid_0's SMAPE: 33.4168
[12]	valid_0's rmse: 188.145	valid_0's SMAPE: 33.4582
[13]	valid_0's rmse: 189.012	valid_0's SMAPE: 33.4885
[14]	valid_0's rmse: 189.602	valid_0's SMAPE: 33.5159
  0%|          | 0/20 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))



[15]	valid_0's rmse: 190.373	valid_0's SMAPE: 33.5477
[16]	valid_0's rmse: 191.318	valid_0's SMAPE: 33.5892
[17]	valid_0's rmse: 192.633	valid_0's SMAPE: 33.6136
[18]	valid_0's rmse: 193.129	valid_0's SMAPE: 33.6511
[19]	valid_0's rmse: 194.036	valid_0's SMAPE: 33.6889
[20]	valid_0's rmse: 195.233	valid_0's SMAPE: 33.7146
[21]	valid_0's rmse: 195.272	valid_0's SMAPE: 33.7408
[22]	valid_0's rmse: 195.152	valid_0's SMAPE: 33.7701
[23]	valid_0's rmse: 196.141	valid_0's SMAPE: 33.7926
[24]	valid_0's rmse: 197.322	valid_0's SMAPE: 33.825
[25]	valid_0's rmse: 198.304	valid_0's SMAPE: 33.8458
[26]	valid_0's rmse: 199.214	valid_0's SMAPE: 33.8749
[27]	valid_0's rmse: 199.644	valid_0's SMAPE: 33.9024
[28]	valid_0's rmse: 200.738	valid_0's SMAPE: 33.9329
[29]	valid_0's rmse: 201.25	valid_0's SMAPE: 33.9512
[30]	valid_0's rmse: 202.108	valid_0's SMAPE: 33.9688
[31]	valid_0's rmse: 203.232	valid_0's SMAPE: 33.9973
[32]	valid_0's rmse: 204.216	valid_0's SMAPE: 34.0254
[33]	valid_0's rmse: 205.111	v

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[12]	valid_0's rmse: 188.176	valid_0's SMAPE: 33.4599
[13]	valid_0's rmse: 189.051	valid_0's SMAPE: 33.4904
[14]	valid_0's rmse: 189.72	valid_0's SMAPE: 33.5209
[15]	valid_0's rmse: 190.51	valid_0's SMAPE: 33.553
[16]	valid_0's rmse: 191.476	valid_0's SMAPE: 33.5947
[17]	valid_0's rmse: 192.813	valid_0's SMAPE: 33.6192
[18]	valid_0's rmse: 193.325	valid_0's SMAPE: 33.657
[19]	valid_0's rmse: 194.247	valid_0's SMAPE: 33.695
[20]	valid_0's rmse: 195.461	valid_0's SMAPE: 33.7208
[21]	valid_0's rmse: 195.513	valid_0's SMAPE: 33.7473
[22]	valid_0's rmse: 195.394	valid_0's SMAPE: 33.7767
[23]	valid_0's rmse: 196.385	valid_0's SMAPE: 33.7992
[24]	valid_0's rmse: 197.578	valid_0's SMAPE: 33.8317
[25]	valid_0's rmse: 198.576	valid_0's SMAPE: 33.8526
[26]	valid_0's rmse: 199.495	valid_0's SMAPE: 33.8817
[27]	valid_0's rmse: 199.926	valid_0's SMAPE: 33.9095
[28]	valid_0's rmse: 201.031	valid_0's SMAPE: 33.9399
[29]	valid_0's rmse: 201.548	valid_0's SMAPE: 33.9584
[30]	valid_0's rmse: 202.34	valid

...

[15]	valid_0's rmse: 187.271	valid_0's SMAPE: 33.4721
[16]	valid_0's rmse: 187.895	valid_0's SMAPE: 33.5079
[17]	valid_0's rmse: 188.605	valid_0's SMAPE: 33.5296
[18]	valid_0's rmse: 188.99	valid_0's SMAPE: 33.5636
[19]	valid_0's rmse: 189.775	valid_0's SMAPE: 33.5972
[20]	valid_0's rmse: 190.608	valid_0's SMAPE: 33.622
[21]	valid_0's rmse: 190.951	valid_0's SMAPE: 33.646
[22]	valid_0's rmse: 190.789	valid_0's SMAPE: 33.6761
[23]	valid_0's rmse: 191.723	valid_0's SMAPE: 33.6963
[24]	valid_0's rmse: 192.697	valid_0's SMAPE: 33.7256
[25]	valid_0's rmse: 193.514	valid_0's SMAPE: 33.7452
[26]	valid_0's rmse: 193.606	valid_0's SMAPE: 33.7735
[27]	valid_0's rmse: 193.694	valid_0's SMAPE: 33.7996
[28]	valid_0's rmse: 194.692	valid_0's SMAPE: 33.8292
[29]	valid_0's rmse: 195.227	valid_0's SMAPE: 33.8471
[30]	valid_0's rmse: 195.949	valid_0's SMAPE: 33.8638
[31]	valid_0's rmse: 196.903	valid_0's SMAPE: 33.8915
[32]	valid_0's rmse: 197.765	valid_0's SMAPE: 33.918
[33]	valid_0's rmse: 198.755	val

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[14]	valid_0's rmse: 186.692	valid_0's SMAPE: 33.4457
[15]	valid_0's rmse: 187.266	valid_0's SMAPE: 33.4717
[16]	valid_0's rmse: 187.888	valid_0's SMAPE: 33.5075
[17]	valid_0's rmse: 188.596	valid_0's SMAPE: 33.5292
[18]	valid_0's rmse: 188.98	valid_0's SMAPE: 33.5631
[19]	valid_0's rmse: 189.764	valid_0's SMAPE: 33.5967
[20]	valid_0's rmse: 190.596	valid_0's SMAPE: 33.6215
[21]	valid_0's rmse: 190.938	valid_0's SMAPE: 33.6456
[22]	valid_0's rmse: 190.776	valid_0's SMAPE: 33.6756
[23]	valid_0's rmse: 191.709	valid_0's SMAPE: 33.6957
[24]	valid_0's rmse: 192.682	valid_0's SMAPE: 33.7251
[25]	valid_0's rmse: 193.498	valid_0's SMAPE: 33.7446
[26]	valid_0's rmse: 193.589	valid_0's SMAPE: 33.773
[27]	valid_0's rmse: 193.676	valid_0's SMAPE: 33.799
[28]	valid_0's rmse: 194.674	valid_0's SMAPE: 33.8286
[29]	valid_0's rmse: 195.208	valid_0's SMAPE: 33.8466
[30]	valid_0's rmse: 195.929	valid_0's SMAPE: 33.8632
[31]	valid_0's rmse: 196.882	valid_0's SMAPE: 33.891
[32]	valid_0's rmse: 197.744	val

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[20]	valid_0's rmse: 192.425	valid_0's SMAPE: 33.6583
[21]	valid_0's rmse: 192.427	valid_0's SMAPE: 33.6833
[22]	valid_0's rmse: 192.312	valid_0's SMAPE: 33.713
[23]	valid_0's rmse: 193.393	valid_0's SMAPE: 33.7337
[24]	valid_0's rmse: 194.501	valid_0's SMAPE: 33.7643
[25]	valid_0's rmse: 195.347	valid_0's SMAPE: 33.7843
[26]	valid_0's rmse: 195.676	valid_0's SMAPE: 33.8136
[27]	valid_0's rmse: 195.97	valid_0's SMAPE: 33.84
[28]	valid_0's rmse: 196.998	valid_0's SMAPE: 33.8695
[29]	valid_0's rmse: 197.581	valid_0's SMAPE: 33.8879
[30]	valid_0's rmse: 198.35	valid_0's SMAPE: 33.9066
[31]	valid_0's rmse: 199.394	valid_0's SMAPE: 33.9351
[32]	valid_0's rmse: 200.292	valid_0's SMAPE: 33.9624
[33]	valid_0's rmse: 201.321	valid_0's SMAPE: 33.9778
[34]	valid_0's rmse: 201.915	valid_0's SMAPE: 33.9925
[35]	valid_0's rmse: 202.921	valid_0's SMAPE: 34.0186
[36]	valid_0's rmse: 203.657	valid_0's SMAPE: 34.0325
[37]	valid_0's rmse: 204.33	valid_0's SMAPE: 34.0555
[38]	valid_0's rmse: 205.017	valid

In [49]:
RMSE = lightgbm_factory(best)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best lightgbm:', np.sqrt(RMSE))

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 194.877	valid_0's SMAPE: 32.9913
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 192.815	valid_0's SMAPE: 33.0403
[3]	valid_0's rmse: 191.159	valid_0's SMAPE: 33.0804
[4]	valid_0's rmse: 189.859	valid_0's SMAPE: 33.1225
[5]	valid_0's rmse: 189.032	valid_0's SMAPE: 33.163
[6]	valid_0's rmse: 188.108	valid_0's SMAPE: 33.2019
[7]	valid_0's rmse: 187.863	valid_0's SMAPE: 33.2328
[8]	valid_0's rmse: 187	valid_0's SMAPE: 33.2707
[9]	valid_0's rmse: 186.462	valid_0's SMAPE: 33.3039
[10]	valid_0's rmse: 186.392	valid_0's SMAPE: 33.3312
[11]	valid_0's rmse: 185.182	valid_0's SMAPE: 33.3601
[12]	valid_0's rmse: 185.812	valid_0's SMAPE: 33.3962
[13]	valid_0's rmse: 186.136	valid_0's SMAPE: 33.4246
[14]	valid_0's rmse: 186.738	valid_0's SMAPE: 33.4508
[15]	valid_0's rmse: 187.333	valid_0's SMAPE: 33.4771
[16]	valid_0's rmse: 187.975	valid_0's SMAPE: 33.5131
[17]	valid_0's rmse: 188.703	valid_0's SMAPE: 33.5351
[18]	valid_0's rmse: 189.099	vali

##  4.填充最优参数并预测0318号的数据，预测SMAPE得分

In [50]:
params = {
        'task': 'train',
        'max_depth':16,
        'num_trees':413,
        'learning_rate':0.051011639607821485,
        'bagging_fraction':0.55 ,
        'num_leaves':85,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'feature_fraction': 0.7,  # 样本列采样
        'lambda_l1': 0,  # L1 正则化
        'lambda_l2': 0,  # L2 正则化
        'bagging_seed': 100,  # 随机种子,light中默认为100
        'silent':1,
        }
print('Start training...')
# train
gbm = lgb.train(params,
                train_data,
                num_boost_round=20,
                valid_sets=test_data,
                early_stopping_rounds=5)
print('Save model...')
# save model to file
gbm.save_model('model.txt')
print('Start predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print(y_pred)

Start training...
[1]	valid_0's l2: 37684.9
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 36607.6
[3]	valid_0's l2: 35818.5
[4]	valid_0's l2: 35079.5
[5]	valid_0's l2: 34532.1
[6]	valid_0's l2: 33993.7
[7]	valid_0's l2: 33752.6
[8]	valid_0's l2: 33403.4
[9]	valid_0's l2: 33018.6
[10]	valid_0's l2: 32626.5
[11]	valid_0's l2: 32028
[12]	valid_0's l2: 32139.7
[13]	valid_0's l2: 32393.7
[14]	valid_0's l2: 32652.3
[15]	valid_0's l2: 32794.4
[16]	valid_0's l2: 33010.9
Early stopping, best iteration is:
[11]	valid_0's l2: 32028
Save model...
Start predicting...
[ 38.39857978  38.13737407  35.85164073 ... 134.65184795  39.83081991
  38.39857978]


/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [51]:
# 测试分数 (0-2之间,越小越好)
def getSMAPEScore (y_true, y_pred):
    #print(y_true)
    # 预测值小于0如何处理 y_pred=np.abs(y_pred)
    y_pred=(np.abs(y_pred) + y_pred) / 2
    SMAPE=2.0 * np.mean(np.abs(y_pred - y_true+0.0001) / (y_pred+ y_true+0.0001))
    return SMAPE,40*(1-SMAPE/2)

In [52]:
getSMAPEScore(y_test,y_pred)

(1.3154657848286717, 13.690684303426565)